<a href="https://colab.research.google.com/github/Akshay-Kumar-Arya/Identify_the_sentiments/blob/master/bert_MLP_model_evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MLP model using Bert vectors

In [1]:
import sklearn
from sklearn.model_selection import train_test_split
import pickle
import pandas as pd
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score

pd.set_option('display.max_colwidth', 200)

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [3]:
# Data path
training_data_path = "/content/gdrive/My Drive/Identify_the_sentiments/train.csv"
test_data_path =  "/content/gdrive/My Drive/Identify_the_sentiments/test.csv"

save_path = "/content/gdrive/My Drive/Identify_the_sentiments/"

In [4]:
# reading data from csv
train_data = pd.read_csv(training_data_path)
test_data = pd.read_csv(test_data_path)

print (train_data.shape, test_data.shape)

(7920, 3) (1953, 2)


In [5]:
# import bert embeddings for tweets
train_file = open(save_path + "bert_train.pickle", "rb")
pool_embs_train, all_embs_train = pickle.load(train_file)

test_file = open(save_path + "bert_test.pickle", "rb")
pool_embs_test, all_embs_test = pickle.load(test_file)

In [7]:
# splitting training and validation data
xtrain, xvalid, ytrain, yvalid = train_test_split(pool_embs_train, 
                                                  train_data['label'],  
                                                  random_state=42, 
                                                  test_size=0.2)
print (xtrain.shape, xvalid.shape, ytrain.shape, yvalid.shape)

(6336, 768) (1584, 768) (6336,) (1584,)


In [55]:
# MLP classifier model
mlp = MLPClassifier(activation='logistic', solver='adam', alpha=1e-5,
                    hidden_layer_sizes=(100,50), random_state=1,
                    learning_rate_init=0.01,max_iter=100, verbose= True, warm_start = True, learning_rate='invscaling')

mlp.fit(xtrain, ytrain)

Iteration 1, loss = 0.56996895
Iteration 2, loss = 0.54411294
Iteration 3, loss = 0.41276841
Iteration 4, loss = 0.33394444
Iteration 5, loss = 0.32141928
Iteration 6, loss = 0.30461050
Iteration 7, loss = 0.28743850
Iteration 8, loss = 0.29816610
Iteration 9, loss = 0.27591259
Iteration 10, loss = 0.28037288
Iteration 11, loss = 0.29894098
Iteration 12, loss = 0.29641502
Iteration 13, loss = 0.26877739
Iteration 14, loss = 0.26687115
Iteration 15, loss = 0.27225133
Iteration 16, loss = 0.26798459
Iteration 17, loss = 0.26265684
Iteration 18, loss = 0.25897409
Iteration 19, loss = 0.25912655
Iteration 20, loss = 0.25805479
Iteration 21, loss = 0.26020400
Iteration 22, loss = 0.25770641
Iteration 23, loss = 0.25097493
Iteration 24, loss = 0.26070072
Iteration 25, loss = 0.25451664
Iteration 26, loss = 0.25999250
Iteration 27, loss = 0.25139548
Iteration 28, loss = 0.24952626
Iteration 29, loss = 0.24904179
Iteration 30, loss = 0.26251954
Iteration 31, loss = 0.25446713
Iteration 32, los

MLPClassifier(activation='logistic', alpha=1e-05, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(100, 50), learning_rate='invscaling',
              learning_rate_init=0.01, max_fun=15000, max_iter=100,
              momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
              power_t=0.5, random_state=1, shuffle=True, solver='adam',
              tol=0.0001, validation_fraction=0.1, verbose=True,
              warm_start=True)

In [56]:
# prediction on validation dataset
preds_valid = mlp.predict(xvalid)
print(f1_score(yvalid, preds_valid))
print(accuracy_score(yvalid, preds_valid))

0.8138075313807531
0.8876262626262627


In [57]:
# prediction on test dataset
preds_test = mlp.predict(pool_embs_test)


# prepare submission dataframe
sub = pd.DataFrame({'id':test_data['id'], 'label':preds_test})

# write predictions to a CSV file
sub.to_csv(save_path + "sub_mlp_bert.csv", index=False)
#Your score for this submission is : 0.8792921986900165.